In [1]:
import numpy as np 
import pandas as pd 
import torch

In [2]:
train_data_path = './data/d_training_set.csv' 
test_data_path = './data/d_test_set.csv' 

train_data = pd.read_csv(train_data_path) 
test_data = pd.read_csv(test_data_path)

In [3]:
# Function to preprocess both datasets 
def process_data(dframe, test=None):
    df = dframe
    print('Data Processing Started')
    if not test: 
        df= df.drop('row_id', axis=1)
        #X, y = df.loc[:, df.columns != 'repaid_loan'], df['repaid_loan'] 
        cat_features = (df.dtypes == 'object') 
        cat_features = list(cat_features[cat_features].index) 
        num_features = (df.dtypes != 'object') 
        num_features = list(num_features[num_features].index) 

        df_cat = df[cat_features] 
        df_num  = df[num_features] 

        df_cat.fillna('Unknown', axis=1, inplace=True)
        # Additional numerical features in the dataset 
        num_features_2 = ['term', 'int_rate', 'emp_length', 'issue_d', 'revol_util'] 

        # Replacing the percentage symbol with an empty string 
        df_cat['int_rate'] = df_cat['int_rate'].replace({'%': ''}, regex=True) 
        # convert the string to a float 
        df_cat['int_rate'] = df_cat['int_rate'].astype(float) 
        # Replacing string symbols with integers 
        df_cat['term'] = df_cat['term'].replace({' 36 months': 36, ' 60 months': 60}, regex=True)
        df_cat['emp_length'] = df_cat['emp_length'].replace({'years': '', '10+': 10, 'year': '', '< 1 year': 1}, regex=True)
        # drop the 'issue_d' column as we deem it unnecessary for the model 
        df_cat.drop('issue_d', axis=1, inplace=True)
        df_cat['revol_util'] = df_cat['revol_util'].replace({'%': ''}, regex=True) 
        df_cat['revol_util'] = df_cat['revol_util'].replace({'Unknown': 0}, regex=True)
        df_cat['revol_util'] = df_cat['revol_util'].astype(float) 
        df_cat['term'] = df_cat['term'].astype(float)
        num_features_2.remove('issue_d')

        df_num_2 = df_cat[num_features_2] 
        df_num = pd.concat([df_num, df_num_2], axis=1)
        # Filling missing variables with the mean of each column 
        df_num.fillna(df_num.mean(), inplace=True) 
        # drop the new numeric features from the categorical dataframe
        df_cat = df_cat.drop(num_features_2, axis=1)

        df_num['emp_length'] = df_num['emp_length'].replace({'< 1': 1}, regex=True) 
        df_num = df_num.replace({'Unknown': 0}, regex=True)

        df_combined = pd.concat([df_num, df_cat], axis=1)
        # categorical features (or attributes) 
        cat_attribs = list(df_cat.columns) 
        num_attribs = list(df_num.columns) 
        if 'repaid_loan' in num_attribs:
            num_attribs.remove('repaid_loan')
        print('*************Training data processed!!*************')
        return df_combined, cat_attribs, num_attribs

    elif test: 
        # if it's the test set 
        if 'row_id' in list(df.columns):
            
            test_row_id = df[['row_id']].copy() 
            df=df.drop('row_id', axis=1)
        cat_features = (df.dtypes == 'object') 
        cat_features = list(cat_features[cat_features].index) 
        num_features = (df.dtypes != 'object') 
        num_features = list(num_features[num_features].index) 

        df_cat = df[cat_features] 
        df_num  = df[num_features] 

        df_cat.fillna('Unknown', axis=1, inplace=True)
        # Additional numerical features in the dataset 
        num_features_2 = ['term', 'int_rate', 'emp_length', 'issue_d', 'revol_util'] 

        # Replacing the percentage symbol with an empty string 
        df_cat['int_rate'] = df_cat['int_rate'].replace({'%': ''}, regex=True) 
        # convert the string to a float 
        df_cat['int_rate'] = df_cat['int_rate'].astype(float) 
        # Replacing string symbols with integers 
        df_cat['term'] = df_cat['term'].replace({' 36 months': 36, ' 60 months': 60}, regex=True)
        df_cat['emp_length'] = df_cat['emp_length'].replace({'years': '', '10+': 10, 'year': '', '< 1 year': 1}, regex=True)
        # drop the 'issue_d' column as we deem it unnecessary for the model 
        df_cat.drop('issue_d', axis=1, inplace=True)
        df_cat['revol_util'] = df_cat['revol_util'].replace({'%': ''}, regex=True) 
        df_cat['revol_util'] = df_cat['revol_util'].replace({'Unknown': 0}, regex=True)
        df_cat['revol_util'] = df_cat['revol_util'].astype(float) 
        df_cat['term'] = df_cat['term'].astype(float)
        num_features_2.remove('issue_d')

        df_num_2 = df_cat[num_features_2] 
        df_num = pd.concat([df_num, df_num_2], axis=1)
        # Filling missing variables with the mean of each column 
        df_num.fillna(df_num.mean(), inplace=True) 
        # drop the new numeric features from the categorical dataframe
        df_cat = df_cat.drop(num_features_2, axis=1)

        df_num['emp_length'] = df_num['emp_length'].replace({'< 1': 1}, regex=True) 
        df_num = df_num.replace({'Unknown': 0}, regex=True)

        df_combined = pd.concat([df_num, df_cat], axis=1)
        # categorical features (or attributes) 
        cat_attribs = list(df_cat.columns) 
        num_attribs = list(df_num.columns) 
        print('*************Test data processed!!***************')
        return df_combined, cat_attribs, num_attribs, test_row_id


In [4]:
X_train_processed, cat_attribs, num_attribs = process_data(train_data) 
X_test_processed, _, _, row_id_col = process_data(test_data, test=True)

Data Processing Started


/home/iron_onet/Documents/Courses/algotrade/venv/lib/python3.8/site-packages/pandas/core/frame.py:5171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/tmp/ipykernel_69032/2912398518.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['int_rate'] = df_cat['int_rate'].replace({'%': ''}, regex=True)
/tmp/ipykernel_69032/2912398518.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

*************Training data processed!!*************
Data Processing Started


/tmp/ipykernel_69032/2912398518.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['int_rate'] = df_cat['int_rate'].replace({'%': ''}, regex=True)
/tmp/ipykernel_69032/2912398518.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['int_rate'] = df_cat['int_rate'].astype(float)
/tmp/ipykernel_69032/2912398518.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

*************Test data processed!!***************


/tmp/ipykernel_69032/2912398518.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['emp_length'] = df_cat['emp_length'].replace({'years': '', '10+': 10, 'year': '', '< 1 year': 1}, regex=True)
/tmp/ipykernel_69032/2912398518.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['revol_util'] = df_cat['revol_util'].replace({'%': ''}, regex=True)
/tmp/ipykernel_69032/2912398518.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [5]:
X_train_num, y_train = X_train_processed[num_attribs], X_train_processed['repaid_loan'] 
X_test_num = X_test_processed[num_attribs]

In [6]:
y_train.head(20)

0     1
1     1
2     1
3     1
4     0
5     1
6     1
7     1
8     1
9     1
10    1
11    0
12    0
13    1
14    1
15    0
16    1
17    1
18    1
19    1
Name: repaid_loan, dtype: int64

In [7]:
X_train_num.head(10)

,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,term,int_rate,emp_length,revol_util
0,10000,10000,10000.0,312.86,40000.0,5.34,1,0.0,16.000000,66.363867,...,0,0,244931,4640,2400,3313,36.0,7.89,9,44.8
1,27500,27500,27500.0,581.60,55000.0,1.32,0,0.0,33.804581,66.363867,...,0,0,339800,2839,32100,0,60.0,9.80,10,6.8
2,20000,20000,20000.0,688.03,153000.0,9.64,0,0.0,33.804581,66.363867,...,0,0,82999,53132,57500,24083,36.0,14.46,7,69.2
3,2900,2900,2900.0,100.98,105000.0,34.61,0,1.0,33.804581,66.363867,...,0,0,476477,124308,33100,108444,36.0,15.31,10,61.2
4,2400,2400,2400.0,87.97,50000.0,1.80,0,5.0,24.000000,14.000000,...,1,0,9900,1400,3000,0,36.0,18.99,2,14.1
5,16000,16000,16000.0,550.43,90000.0,14.77,0,3.0,33.804581,66.363867,...,0,0,218964,45851,8300,34326,36.0,14.46,1,61.7
6,35000,35000,35000.0,888.20,148000.0,19.01,0,1.0,61.000000,43.000000,...,1,8,124956,90623,28200,89756,60.0,17.97,3,70.0
7,10225,10225,10225.0,232.60,83000.0,14.50,0,0.0,33.804581,66.363867,...,0,0,193223,41836,19500,41752,60.0,12.99,2,32.6
8,17500,17500,17500.0,540.27,73000.0,11.38,0,1.0,41.000000,66.363867,...,0,0,208911,31337,21400,34521,36.0,6.99,10,78.1
9,19800,19800,19800.0,524.47,89000.0,25.49,0,1.0,33.804581,66.363867,...,0,0,455019,73568,41300,61142,60.0,19.99,9,25.4


In [11]:
X_train_num.values

array([[10000, 10000, 10000.0, ..., 7.89, '9 ', 44.8],
       [27500, 27500, 27500.0, ..., 9.8, 10, 6.8],
       [20000, 20000, 20000.0, ..., 14.46, '7 ', 69.2],
       ...,
       [16000, 16000, 16000.0, ..., 13.44, 10, 15.9],
       [20000, 20000, 20000.0, ..., 13.67, 10, 72.8],
       [15000, 15000, 15000.0, ..., 7.89, '3 ', 62.9]], dtype=object)

In [12]:
X_train_array = X_train_num.values
Y_train_array = y_train.values

In [ ]:
import torch 
from torch.utils import DataLoader 

